# Wine Classification

### Importo le librerie

In [9]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pandas as pd
import numpy as np
import tensorflow as tf
from tinymlgen import port
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

tf.random.set_seed(1337)
np.random.seed(1337)

### Carico il Dataset ed eseguo lo split

In [12]:
# load and split dataset into train, validation, test
x_values, y_values = load_wine(return_X_y=True)

print("Total samples: " + str(x_values.shape[0]))

y = tf.keras.utils.to_categorical(y_values)
x_train, x_test, y_train, y_test = train_test_split(x_values, y, test_size=0.2)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3)

print("Train samples: " + str(x_train.shape))
print("Validation samples: " + str(x_valid.shape))
print("Test samples: " + str(x_test.shape))

input_dim = x_train.shape[1:]
output_dim = y.shape[1]

print("Input dimension: ", input_dim)
print("Output dimension: ", output_dim)


Total samples: 178
Train samples: (99, 13)
Validation samples: (43, 13)
Test samples: (36, 13)
Input dimension:  (13,)
Output dimension:  3


### Visualizzo il Dataset

In [3]:
data = load_wine()

# Converto data in un Dataframe per visualizzarlo correttamente
df = pd.DataFrame(data=data["data"], columns=data["feature_names"])
# Aggiungo la rispettiva classe di ogni sample
df["class"] = data["target"]

# Stampo i primi 5 sample
df.head()


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


### Creazione del modello

In [4]:
# creo e alleno il modello
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=20, activation="relu", input_shape=input_dim))
model.add(tf.keras.layers.Dense(units=40, activation="relu"))
model.add(tf.keras.layers.Dense(output_dim, activation="softmax"))

# uso categorical_crossentropy per la classificazione multi classe
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=100)


Epoch 1/100
4/4 [==============================] - 1s 98ms/step - loss: 322.7955 - accuracy: 0.4141 - val_loss: 295.3567 - val_accuracy: 0.3953
Epoch 2/100
4/4 [==============================] - 0s 16ms/step - loss: 265.2348 - accuracy: 0.4141 - val_loss: 235.3179 - val_accuracy: 0.3953
Epoch 3/100
4/4 [==============================] - 0s 15ms/step - loss: 210.4118 - accuracy: 0.4141 - val_loss: 176.2215 - val_accuracy: 0.3953
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 155.1409 - accuracy: 0.4141 - val_loss: 117.8649 - val_accuracy: 0.3953
Epoch 5/100
4/4 [==============================] - 0s 16ms/step - loss: 101.2766 - accuracy: 0.4141 - val_loss: 60.8646 - val_accuracy: 0.3953
Epoch 6/100
4/4 [==============================] - 0s 16ms/step - loss: 48.9670 - accuracy: 0.4141 - val_loss: 21.2679 - val_accuracy: 0.3721
Epoch 7/100
4/4 [==============================] - 0s 15ms/step - loss: 30.1915 - accuracy: 0.3030 - val_loss: 30.7464 - val_accuracy: 0.32

### Accuratezza del modello

In [5]:
print("Accuracy: %.3f" % model.evaluate(x_test, y_test)[1])

2/2 [==============================] - 0s 9ms/step - loss: 0.8528 - accuracy: 0.7778
Accuracy: 0.778


### Seleziono un sample dal Dataset di test da salvare nel file header

In [6]:
cpp_dataset_variables = ""

# seleziono il primo sample del test set
first_sample_x = x_test[0]
first_sample_y = y_test[0]

cpp_dataset_variables = "float x_test[13] = "
x_values_str = str([ x for x in first_sample_x ])
x_values_str = x_values_str.replace("[", "{ ")
x_values_str = x_values_str.replace("]", " };\n")

cpp_dataset_variables += x_values_str

# seleziono l'indice dove è presente il valore 1. nell'array delle classi y reali
cpp_dataset_variables += "const int y_test = " + str(np.where(first_sample_y == 1.)[0][0]) + ";"

print(cpp_dataset_variables)


float x_test[13] = { 13.05, 1.77, 2.1, 17.0, 107.0, 3.0, 3.0, 0.28, 2.03, 5.04, 0.88, 3.35, 885.0 };
const int y_test = 0;


### Salvo il modello nel file header

In [7]:
cpp_code = port(model, variable_name="WineClassificationModel", pretty_print=True, optimize=False)

cpp_code += "\n"
cpp_code = cpp_code + cpp_dataset_variables

# cpp files
with open("assets/cpp_files/WineClassification.h", "w+") as f:
    f.write(cpp_code)
    
# esp32
with open("../WineClassification/WC_ESP32/include/WineClassification.h", "w+") as f:
    f.write(cpp_code)

# esp8266
with open("../WineClassification/WC_ESP8266/include/WineClassification.h", "w+") as f:
    f.write(cpp_code)

# wemos
with open("../WineClassification/WC_WEMOS/include/WineClassification.h", "w+") as f:
    f.write(cpp_code)

INFO:tensorflow:Assets written to: /var/folders/k6/0tv_bzp937scnn3d7yzhy1vr0000gn/T/tmpx0cpgfxy/assets


INFO:tensorflow:Assets written to: /var/folders/k6/0tv_bzp937scnn3d7yzhy1vr0000gn/T/tmpx0cpgfxy/assets


### Previsione di un singolo sample

In [8]:
print(model.predict(x_test[0:1]))
print(y_test[0:1])


1/1 [==============================] - 1s 684ms/step
[[0.63433325 0.09378678 0.27187997]]
[[1. 0. 0.]]
